#### 함수 객체를 통해 전략패턴 리팩토링, 명령패턴 단순화

# 6.1 사례 전략 패턴의 리팩토링
- 전략 패턴은 함수를 일급 객체로 사용함으로써 간단해 질 수 있다

## 6.1.1 고전적인 전략
- 전략 패턴 : 알고리즘을 정의, 각각을 하나의 클래스 안에 넣어 교체하기 쉽게함
- 전략 패턴 구성요소 
- 1. 콘텍스트 : 일부 계산을 다른 알고리즘을 구현하는 컴포넌트에 위임하여 서비스 제공
- 2. 전략 : 여러 알고리즘을 구현하는 컴포넌트의 공통 인터페이스
- 3. 구체적인 전략 : 전략의 구상 서브클래스

### 전략 패턴 예제
- 온라인 상정의 할인 규칙
- 포인트 1000점 이상인 고객은 5% 할인
- 하나의 주분에서 20개 이상 같은제품 구입시, 해당사품 10% 할인
- 서로다른 상품 10종류 이상 주문, 7% 할인

In [8]:
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

jack = Customer('Jack',100)
jack

Customer(name='Jack', fidelity=100)

In [9]:
class LineItem:
    """구매할 물품/갯수 생성해서 총 가격을 반환"""
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
        
    def total(self):
        return self.price * self.quantity


class Order: # Context
    """고객(namedtuple) 및 LineItem 클래스의 인스턴스들을 cart로 받아서 총 계산할 가격을 산출"""
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion # 할인 객체
        
    def total(self):
        """__total 속성이 없으면 전체 계산할 값을 계산"""
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    
    def due(self):
        """할인금액 차감"""
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self) # self = Order 객체
        return self.total() - discount
    
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

In [10]:
# 전략 구조 생성
class Promotion(ABC): # strategy : abstract base class 
    """할인 혜택 클래스들의 형태를 선언"""
    
    @abstractmethod # 이 클래스를 상속하는 클래스는 반드시 이 메서드를 선언해야함
    def discount(self, order):
        """할인액을 구체적인 숫자로 반환"""
        pass

# 구체적인 전략 생성
class FidelityPromo(Promotion):
    """충성도 점수가 1000점 이상인 고객에게 전체 5% 할인 적용"""
    
    def discount(self, order):
        return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

class BulkItemPromo(Promotion): 
    """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""
    
    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * 0.1
        return discount
    
class LargeOrderPromo(Promotion):
    """10종류 이상의 상품을 구입하면 전체 7% 할인 적용"""
    
    def discount(self, order):
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * 0.07

In [11]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, .5),
       LineItem('apple', 10, 1.5),
       LineItem('watermellon', 5, 5.0)]

Order(joe, cart, FidelityPromo()) # 충성도 할인 없음

<Order total: 42.00 due: 42.00>

In [12]:
Order(ann, cart, FidelityPromo()) # 충성도 할인 적용

<Order total: 42.00 due: 39.90>

In [13]:
banana_cart = [LineItem('banana', 30, .5),
               LineItem('apple', 10, 1.5)]
Order(joe, banana_cart, BulkItemPromo()) # 바나나 20개 이상 --> 10% 할인

<Order total: 30.00 due: 28.50>

In [14]:
long_order = [LineItem(str(item_code), 1, 1.0)
             for item_code in range(10)] # 물품 종류가 10개
Order(joe, long_order, LargeOrderPromo()) # 10개 이상 10% 할인

<Order total: 10.00 due: 9.30>

## 6.1.2 함수지향 전략
- 앞선 예제의 구체적 전략 객체를 함수로 대체

In [15]:
class Order: # Context
    """고객(namedtuple) 및 LineItem 클래스의 인스턴스들을 cart로 받아서 총 계산할 가격을 산출"""
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion # 할인 객체
        
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self) # promotion은 함수 객체
        return self.total() - discount
    
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

In [16]:
#개별 전략을 함수로 정의
def fidelity_promo(order):
    """충성도 점수가 1000점 이상인 고객에게 전체 5% 할인 적용"""
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

def bulk_item_promo(order): 
    """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * 0.1
    return discount
    
def large_order_promo(order):
    """10종류 이상의 상품을 구입하면 전체 7% 할인 적용"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * 0.07
    return 0

In [17]:
Order(ann, cart, fidelity_promo)

<Order total: 42.00 due: 39.90>

In [18]:
Order(joe, banana_cart, bulk_item_promo), \
Order(ann, long_order, large_order_promo)


(<Order total: 30.00 due: 28.50>, <Order total: 10.00 due: 9.30>)

### 플라이웨이트
- 여러 콘텍스트에서 동시에 사용할 수 있는 공유 객체 
- 전략 객체를 반복하는 경우, 플라이웨이트 공유가 생성 비용을 절약함
- 전략 객체가 상태가 없는 경우, 함수로 만드는것이 나음 

## 6.1.3 최선의 전략 선택 : 단순한 접근법
- Order 객체에 대해 적용할 수 있는 가장 좋은 할인 전략을 선택하는 '메타 전략'을 만든다고 가정 

In [19]:
promos = [fidelity_promo, bulk_item_promo, large_order_promo]
# 함수들로 구현된 전략들의 리스트

def best_promo(order):
    """최대로 할인받을 금액을 반환"""
    return max(promo(order) for promo in promos)

In [20]:
Order(joe, long_order, best_promo),\
Order(joe, banana_cart, best_promo),\
    Order(ann, cart, best_promo)

(<Order total: 10.00 due: 9.30>,
 <Order total: 30.00 due: 28.50>,
 <Order total: 42.00 due: 39.90>)

In [21]:
# 일부 코드 중복으로 버그 발생 가능성 
# - 새로운 할인 전략 추가시, 함수를 promos 리스트에 추가
# - 새로운 할인 함수를 Order 객체에 인수로 전달할 수 있으나, best_promo()는 새로운 할인 함수를 고려하지 않음

## 6.1.4 모듈에서 전략 찾기 
- globals() : 모듈을 다루는 함수, 전역 심벌 테이블을 나타내는 딕셔너리 반환 
- 함수가 정의된 모듈을 찾을 수 있다.

In [22]:
globals().keys()

dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', '_', '__', '___', 'os', 'sys', '_i', '_ii', '_iii', '_i1', 'Order', '_i2', 'fidelity_promo', 'bulk_item_promo', 'large_order_promo', '_i3', 'promos', 'best_promo', '_i4', '_i5', '_i6', '_i7', '_i8', 'ABC', 'abstractmethod', 'namedtuple', 'Customer', 'jack', '_8', '_i9', 'LineItem', '_i10', 'Promotion', 'FidelityPromo', 'BulkItemPromo', 'LargeOrderPromo', '_i11', 'joe', 'ann', 'cart', '_11', '_i12', '_12', '_i13', 'banana_cart', '_13', '_i14', 'long_order', '_14', '_i15', '_i16', '_i17', '_17', '_i18', '_18', '_i19', '_i20', '_20', '_i21', '_i22'])

In [24]:
globals()['_i17']

'Order(ann, cart, fidelity_promo)'

In [25]:
#  *_promo() 함수 찾기
promos = [globals()[name] for name in globals()
         if name.endswith('_promo')
         and name != 'best_promo'] # 재귀호출을 피하기 위함
promos

[<function __main__.fidelity_promo(order)>,
 <function __main__.bulk_item_promo(order)>,
 <function __main__.large_order_promo(order)>]

In [26]:
import promotions, inspect
#promotions.py

promos = [func for name, func in
         inspect.getmembers(promotions, inspect.isfunction)]

promos

[<function promotions.bulk_item_promo(order)>,
 <function promotions.fidelity_promo(order)>,
 <function promotions.large_order_promo(order)>]

# 6.2 명령
- 함수를 인자로 전달하여 명령 디자인 패턴도 단순하게 만들 수 있다
- 명령 패턴 : 연산을 실행하는 객체(invoker)와 연산을 수행하는 객체(receiver)를 분리하는 패턴

In [27]:
#명령 패턴 예제
# 구글홈을 사용하는 사용자, 구글홈, 히터 
class Heater:
    def __init__(self):
        pass
    
    def powerOn(self):
        print("Heater on")

class OKGoogle:
    def __init__(self, heater):
        self.heater = heater
        
    def talk(self):
        self.heater.powerOn()

class Client:
    def __init__(self):
        pass
    
    def main(self):
        heater = Heater()
        okGoogle = OKGoogle(heater)
        okGoogle.talk()

In [28]:
client = Client()
client.main()

Heater on


In [29]:
#램프 켜기 추가 
# Lamp class 정의, OKGoogle 램프참조

class Lamp:
    def __init__(self):
        pass
    
    def turnOn(self):
        print("Lamp on")


modes = ['heater', 'lamp']


class OKGoogle:
    def __init__(self, heater, lamp):
        self.heater = heater
        self.lamp = lamp
        
    def setMode(self, idx):
        self.mode = modes[idx]
        
    def talk(self):
        if self.mode == 'heater':
            self.heater.powerOn()
        elif self.mode == 'lamp':
            self.lamp.turnOn()


class Client:
    def __init__(self):
        pass
    
    def main(self):
        heater = Heater()
        lamp = Lamp()
        okGoogle = OKGoogle(heater, lamp)
        
        # 히터 켜짐
        okGoogle.setMode(0)
        okGoogle.talk()
        
        # 램프 켜짐
        okGoogle.setMode(1)
        okGoogle.talk()

client = Client()
client.main()

Heater on
Lamp on


In [30]:
# OKGoogle에게 mode 설정을 통해 mode='heater'이면 히터, mode='lamp'이면 램프를 켜도록 설정
# 기능이 많아질 수록 객체의 프로퍼티는 더욱 늘어나고 talk() 메서드의 분기 증가
# OCP("클래스나 모듈은 확장에는 열려 있어야 하고 변경에는 닫혀 있어야 한다") 원칙에도 위배
# 해결 --> 기능의 캡슐화, talk()메서드가 캡슐화한 인터페이스의 메서드를 호출

In [32]:
# 인터페이스 정의
class Command(ABC):

    @abstractmethod
    def run(self):
        pass


# 명령들을 각각 클래스화
class HeaterOnCommand(Command):
    def __init__(self, heater):
        self.heater = heater
        
    def run(self):
        self.heater.powerOn()

class Heater:
    
    def powerOn(self):
        print("Heater on")


class LampOnCommand(Command):
    def __init__(self, lamp):
        self.lamp = lamp
        
    def run(self):
        self.lamp.turnOn()

class Lamp:
    
    def turnOn(self):
        print("Lamp on")


# OKGoogle 클래스의 talk() 메서드에서는 Command 인터페이스의 run() 메서드를 통해 명령을 실행
class OKGoogle:
    
    def setCommand(self, command): #커멘드 함수를 인자로 받음
        self.command = command
        
    def talk(self):
        self.command.run()

# OKGoogle을 사용하는 Client 클래스를 정의
class Client:
    def __init__(self):
        pass
    
    def main(self):
        heater = Heater()
        lamp = Lamp()
        
        heaterOnCommand = HeaterOnCommand(heater)
        lampOnCommand = LampOnCommand(lamp)
        okGoogle = OKGoogle()
        
        # 히터를 켠다
        okGoogle.setCommand(heaterOnCommand)
        okGoogle.talk()
        
        # 램프를 켠다
        okGoogle.setCommand(lampOnCommand)
        okGoogle.talk()

client = Client()
client.main()

Heater on
Lamp on


In [33]:
#  HeaterOnCommand 클래스의 run()을 콜러블로 변경
class Command(ABC):

    @abstractmethod
    def __call__(self):
        pass


class HeaterOnCommand(Command):
    def __init__(self, heater):
        self.heater = heater
        
    def __call__(self):
        self.heater.powerOn()

class LampOnCommand(Command):
    def __init__(self, lamp):
        self.lamp = lamp
        
    def __call__(self):
        self.lamp.turnOn()


class OKGoogle:
    def __init__(self):
        pass
    
    def setCommand(self, command):
        self.command = command
        
    def talk(self):
        self.command() # callable 로 변경

class Client:
    def __init__(self):
        pass
    
    def main(self):
        heater = Heater()
        lamp = Lamp()
        
        heaterOnCommand = HeaterOnCommand(heater)
        lampOnCommand = LampOnCommand(lamp)
        okGoogle = OKGoogle()
        
        # 히터를 켠다
        okGoogle.setCommand(heaterOnCommand)
        okGoogle.talk()
        
        # 램프를 켠다
        okGoogle.setCommand(lampOnCommand)
        okGoogle.talk()

client = Client()
client.main()

Heater on
Lamp on
